# Automação Web (selenium)

- Projeto de automação web que iráautomatizar pesquisa de preços com base em uma planilha de produtos, para esse projeto iremos utilizar a biblioteca Selenium do python.

# Entendimento do negócio

- Um funcionário trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Esse funcionário constantemente busca nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

# Objetivo

- Seu objetivo: Criar uma pesquisa automática que, irá buscar por produtos com valor abaixo de mercado, valor definido por você, então deve-se criar uma planilha com esses produtos, indicando o preço, nome do produto e link para compra.

- Em seguida, vai enviar um e-mail com a lista destes produtos para o setor de compra.

- OBS: No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites. Essa busca também pode ser feita por APIs.


### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.


# Importando bibliotecas

In [85]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import pandas as pd


# Coleta de dados (PC)

In [86]:
tabela_prod = pd.read_excel("buscas.xlsx")
display(tabela_prod)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


# Coleta de dados (WEB)

In [87]:
# criando o navegador
nav = webdriver.Chrome()
nav.get("https://www.google.com.br/")

# pegar produto a pesquisar da tabela
produto = "iphone 12 64 gb"
termos_banidos = "mini watch"
# tratar as palavras para deixa-las minusculas
produto = produto.lower()
termos_banidos = termos_banidos.lower()

# criando lista de termos banidos (split remove espacos e cria lista com nomes)
lista_term_banidos = termos_banidos.split(' ')
lista_term_prod = produto.split(' ')
preco_min = 3000
preco_max = 3500

# Vai no input e digita o produto
nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(produto) 
# dar um enter
nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(Keys.ENTER) 
# clicar em shopping: para shopping vamos pegar todos elementos
# da classe "hdtb-mitem" e procurar por "shopping" usando "elementS"
elementos = nav.find_elements(By.CLASS_NAME, "hdtb-mitem")
for item in elementos:
    if "Shopping" in item.text:
        item.click()
        break #paramos pois pode ter mais de um item

# pegar preços, nome e link, pra isso vamos buscar dentro de uma
# div que contém todos esses elementos

# elemento div
lista_resultado = nav.find_elements(By.CLASS_NAME, "sh-dgr__grid-result")

 





In [90]:
# dentro de cada div procura os elementos
for resultado in lista_resultado:
    nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
    # so deixando em minusculo para comparação
    nome = nome.lower()
    
    #criar um filtro para os termos banidos
    tem_termos_banidos = False
    for palavra in lista_term_banidos:
        if palavra in nome:
            tem_termos_banidos = True  
            
    tem_todos_termos_produto = True
    for palavra in lista_term_prod:
        if palavra not in nome:
            tem_todos_termos_produto = False
    
    if tem_termos_banidos == False and tem_todos_termos_produto == True:
        # se tem_termos_banidos = false e tem_todos _termos_produto = true continua
        # tratar o preço para comparação
        preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
        preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
        if preco != "":
            preco = float(preco)
            
        # verificar se o preço está na faixa:
        preco_max = float(preco_max)
        preco_min = float(preco_min)
        if preco_min <= preco <= preco_max:      
            # pelo modo abaixo nao deu certo pegar o link, vamos para outra forma
            # link = resultado.find_element(By.CLASS_NAME, 'shntl').get_attribute('href')
            # entao vamos pegar o link pelo PARENT de outro elemento
            elemento_link = resultado.find_element(By.CLASS_NAME,'aULzUe')
            # XPATH ".." é o elemento pai que está acima do filho
            elemento_pai = elemento_link.find_element(By.XPATH,'..')
            link = elemento_pai.get_attribute('href')
            print(preco, nome, link)

3500.0 iphone 12 64gb azul menos de 1 ano de uso https://www.google.com.br/url?url=https://mg.olx.com.br/belo-horizonte-e-regiao/celulares/iphone-12-64gb-azul-menos-de-1-ano-de-uso-1124387369&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjxk5qehK38AhUtIbkGHfqHCfsQgOUECJUM&usg=AOvVaw1yWCIHZydIaedyqrCKrla5
3329.1 iphone 12 64gb - azul - estou zerado https://www.google.com.br/url?url=https://www.trocafy.com.br/iphone-12-64gb-azul-sou-como-novo-2552/p%3Fidsku%3D966%26srsltid%3DAeTuncoqiW45hsTptbwUx9doQc0nmoqN5395GhyvNvsrZoEXlrL4eZD1HzY&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjxk5qehK38AhUtIbkGHfqHCfsQguUECKYM&usg=AOvVaw3SYgjd6dyKz1xK9HkQ8LUN
3399.15 (seminovos) iphone 12 apple azul, 64gb https://www.google.com.br/url?url=https://www.taqi.com.br/seminovo-iphone12-apple-azul64gb-nacional/220673%3Fsrsltid%3DAeTuncoUV_2YQMFzucCLjQyMSbdgPNA-0sUfFUbRY46XSNHNlM52uL1J-U4&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwjxk5qehK38AhUtIbkGHfqHCfsQgOUECJAP&usg=AOvVaw3IKH7z0ltpgMbprm5_sM8d
3399.15 (seminovos) iphone 12 apple preto, 64gb ht